In [6]:
import feedparser;
import pika;
import json;

In [2]:
rss_url = 'https://rss.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml'
feed = feedparser.parse(rss_url)

In [3]:
output = {
    'id':feed.entries[0].id,
    'title':feed.entries[0].title,
    'url':feed.entries[0].link,
    'author':feed.entries[0].author,
    'date':feed.entries[0].published
}

In [11]:
connection = pika.BlockingConnection(pika.ConnectionParameters('192.168.0.202'))
channel = connection.channel()

In [7]:
json.dumps(output)

'{"id": "https://www.nytimes.com/2025/03/22/world/asia/trump-china-xi-daines.html", "title": "A Montana Senator Seeks to Be Trump\\u2019s Voice in Beijing", "url": "https://www.nytimes.com/2025/03/22/world/asia/trump-china-xi-daines.html", "author": "Keith Bradsher and Meaghan Tobin", "date": "Sat, 22 Mar 2025 13:01:58 +0000"}'

In [8]:
channel.basic_publish(exchange='news.rss', routing_key='nytime', body=json.dumps(output))


In [15]:
channel.basic_consume(queue='nytime',
                      auto_ack=True,
                      on_message_callback=lambda ch, method, properties, body:
                          print(f" [x] Received {json.loads(body)}")
                      )

'ctag1.c95ee5bf685e43e3ba7c6d1b921ba759'

In [16]:
channel.start_consuming()

 [x] Received b'Hello World!'
 [x] Received b'{"id": "https://www.nytimes.com/2025/03/22/world/asia/trump-china-xi-daines.html", "title": "A Montana Senator Seeks to Be Trump\\u2019s Voice in Beijing", "url": "https://www.nytimes.com/2025/03/22/world/asia/trump-china-xi-daines.html", "author": "Keith Bradsher and Meaghan Tobin", "date": "Sat, 22 Mar 2025 13:01:58 +0000"}'


KeyboardInterrupt: 